In [1]:
#This is a piece of code for merging past GMWS class matriculation / post-graduate plans / guidance summary (Word/Excel documents)

#How to use: 
# 1. download 'GMWS_Alumni_Data.csv' and copy it into the same directory as this program is at
#               (note: make sure you have the graduating students' basic info in the csv doc first)
# 2. put all files you want to add to existing data in one folder
#               (note: each student's full name and post-graduate plan should be in the same row, separated by tabs, not spaces)
# 3. copy the file path by right-clicking on the folder and paste it into the function
# 4. run the code, check the items failed to load in, and manually add them into the csv document
# 5. there is an extra function for you to label colleges with other excel/csv files (of college names & their type,rank,location,etc.)

#The source data will be available on DynamoDB in GMWS's AWS account, or you can find it with Yang (huyang2022@126.com)
#more information: see https://github.com/YoungHu2023/GMWS-Connect

import pandas as pd
import zipfile
import xlrd
import openpyxl
import xml.etree.ElementTree as ET
import os

pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

path = "Downloads\GMWS Graduates" #put your folder path here
resource = r"GMWS_Alumni_Data.csv" #put the source data here
main_data = pd.read_csv(resource,encoding='unicode_escape',dtype=str) #put the source data here
graduates = pd.DataFrame(dtype=str)

for root_dir,sub_dir,files in os.walk(path):
    for file in files:
        if file.endswith(".docx"):
            #construct absolute path
            file_name = os.path.join(root_dir, file)

            doc = zipfile.ZipFile(file_name).read('word/document.xml')
            root = ET.fromstring(doc)

            ET.tostring(root)

            ns = {'w': 'http://schemas.openxmlformats.org/wordprocessingml/2006/main'}
            body = root.find('w:body', ns)  # find the XML "body" tag
            p_sections = body.findall('w:p', ns)  # under the body tag, find all the paragraph sections

            for p in p_sections:
                text_elems = p.findall('.//w:t', ns)
                #print('&'.join([t.text for t in text_elems]))
                data = pd.DataFrame('$'.join([t.text for t in text_elems]).split(r'$')).T
                graduates = pd.concat([graduates,data])
                
        if file.endswith(".xlsx"):
            print('This program is not able to deal with excel,excel document(s) are skipped!')
            #I HAVE NOT WRITTEN THIS PART YET
            
graduates = pd.concat([graduates[0], 
             graduates[1].combine_first(graduates[2]).combine_first(graduates[3]).combine_first(graduates[4]).combine_first(graduates[5]).combine_first(graduates[6])], 
            axis=1)
graduates.rename(columns={"1":"Post-graduate Plans"}, inplace=True)
graduates.rename(columns = {0:'User Name',1:'Post-graduate Plans'}, inplace = True)

FileNotFoundError: [Errno 2] No such file or directory: 'GMWS_Alumni_Data.csv'

In [2]:
pd.merge(main_data,graduates,
how='left', on='User Name')

main_data.rename(columns={"Post-graduate Plans_x":"Post-graduate Plans"}, inplace=True)
for index,row in main_data.iterrows():

    if (row['Post-graduate Plans'] is not None):
        main_data.loc[index,'Post-graduate Plans'] = row['Post-graduate Plans']
    elif (row['Post-graduate Plans'] is None):
        main_data.loc[index,'Post-graduate Plans'] = row['Post-graduate Plans_y']
        
#main_data.drop(columns='Post-graduate Plans_y',axis=1,inplace=True)

main_data = main_data.drop(columns='Unnamed: 0',axis=1)

print('The following data is failed to load into your main chart:')
manual_work = pd.merge(graduates, main_data, how='outer', 
indicator=True).query('_merge == "left_only"').drop(columns=['_merge']).dropna(axis=1,how='all').dropna(how='all')
manual_work

KeyError: "['Unnamed: 0'] not found in axis"

In [47]:
main_data = pd.read_csv(r"C:\Users\Yang\Downloads\GMWS_Alumni_Data.csv",encoding='unicode_escape',dtype={'User Name': str,'Post-graduate Plans': str})
#add labels for college using xlsx/csv files (first column must be college names)
#it returns the merged data
def label_colleges(file_path):
    global main_data
    if file_path.endwith(".csv"):
        new_label = pd.read_csv(file_path, dtype=str)
        
    elif file_path.endwith(".xlsx"):
        new_label = pd.read_xlsx(file_path, dtype=str)
    
    else:
        print('The file input is invalid!')
    new_label.rename(columns = {0:'Post-graduate Plans'}, inplace = True)
    main_data = pd.merge(main_data,new_label,
    how='left', on='Post-graduate Plans')
    main_data.drop(columns='Unnamed: 0',axis=1)
    main_data.drop(columns='Unnamed: 0',axis=1)
    return main_data